In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import datetime as dt

import csv



# Read data

In [2]:
# Read data
df = pd.read_csv('../rawdata/traffic-CollisionRecords.csv')
print df.shape

(59368, 76)


/Users/Jay/miniconda3/envs/sf-walk-safety/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (26,29,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Convert date and time to datetime objects

In [3]:
df['COLLISION_DATE'] = df['COLLISION_DATE'].apply(lambda x: dt.datetime.strptime(str(x).zfill(8),"%Y%m%d").date())

df['COLLISION_DATE'].head(3)

0    2003-01-04
1    2003-01-01
2    2003-02-07
Name: COLLISION_DATE, dtype: object

In [4]:
df['COLLISION_TIME'] = df['COLLISION_TIME'].apply(lambda x: dt.datetime.strptime(str(x % 2400).zfill(4),"%H%M").time())

df['COLLISION_TIME'].head(3)

0    12:05:00
1    23:38:00
2    08:00:00
Name: COLLISION_TIME, dtype: object

## Drop non-pedestrian accidents

In [5]:
# Drop non-pedestrian accidents
df = df[pd.notnull(df['PEDESTRIAN_ACCIDENT'])].copy()
df = df[df['PED_ACTION'] != 'A'].copy()

print df.shape


(10459, 76)


## Join data with geocoded lat/long

In [6]:
# Read geocode latlong
geocode = pd.read_csv('../scripts/data_geocoded.txt',sep='|',header=None,names=['CASE_ID','PRIMARY_RD','SECONDARY_RD','y','x'])
print geocode.shape

# Join data
df.reset_index(inplace=True)
df = df.merge(geocode[['CASE_ID','x','y']],how="inner",left_on="CASE_ID",right_on="CASE_ID")
print df.shape

(10479, 5)
(10459, 79)


# Merge weather

In [7]:
def merge_weather(w1,w2):
    
    Weather_Decode = {
        'A' : 'Clear',
        'B' : 'Cloudy',
        'C' : 'Raining',
        'D' : 'Snowing',
        'E' : 'Fog',
        'F' : 'Other',
        'G' : 'Wind',
        '-' : None
    }
    
    out = ''
    
    if Weather_Decode[w1]:
    
        out = Weather_Decode[w1]
        
        if Weather_Decode[w2]:
            out = ', '.join([out, Weather_Decode[w2]])
    
    elif Weather_Decode[w2]:
        out = Weather_Decode[w2]
    
    return out

In [8]:
df['WEATHER'] = df[['WEATHER_1','WEATHER_2']].apply(lambda row : merge_weather(row[0],row[1]), axis=1)
df['WEATHER'].head(3)

0      Clear
1    Raining
2     Cloudy
Name: WEATHER, dtype: object

## Decode day of week

In [9]:
Day_of_Week_Decode = {
    1 : 'Monday',
    2 : 'Tuesday',
    3 : 'Wednesday',
    4 : 'Thursday',
    5 : 'Friday',
    6 : 'Saturday',
    7 : 'Sunday'
}

df['DAY_OF_WEEK'] = df['DAY_OF_WEEK'].apply(lambda x : Day_of_Week_Decode[x])

df['DAY_OF_WEEK'].head(3)

0      Monday
1    Thursday
2      Friday
Name: DAY_OF_WEEK, dtype: object

## Merge location

In [10]:
def merge_location(S):
    
    Direction_Decode = {
        'N' : 'north',
        'W' : 'west',
        'E' : 'east',
        'S' : 'south'
    }
    
    loc = ' and '.join([S['PRIMARY_RD'].strip(), S['SECONDARY_RD'].strip()])
    
    if S['INTERSECTION'] == 'Y':
        loc = ', '.join([loc, 'at intersection'])
    elif not pd.isnull(S['DISTANCE']) and not pd.isnull(S['DIRECTION']): 
        direction = ' ft '.join([str(S['DISTANCE']), Direction_Decode[S['DIRECTION']] ])
        loc = ', '.join([loc, direction])
    
    return loc

In [11]:
df['location'] = df[['PRIMARY_RD','SECONDARY_RD','INTERSECTION','DISTANCE','DIRECTION']].apply(merge_location,axis=1)
df['location'].head()

0       JACKSON ST and GRANT AV, 77.0 ft east
1        TARAVAL ST and 19TH AV, 64.0 ft west
2       N 7TH ST and MARKET ST, 20.0 ft north
3    FRANKLIN ST and POST ST, at intersection
4      CLEMENTINA ST and 4TH ST, 90.0 ft west
Name: location, dtype: object

## Primary Collision Factor

In [12]:
PCF_Decode = {
    '01' : "Driving or bicycling under the influence of alcohol or drug",
    '02' : "Impeding traffic",
    '03' : "Unsafe speed",
    '04' : "Following too closely",
    '05' : "Wrong side of road",
    '06' : "Improper passing",
    '07' : "Unsafe lane change",
    '08' : "Improper turning",
    '09' : "Automobile right of way",
    '10': "Pedestrian right of way",
    '11': "Pedestrian violation",
    '12': "Traffic signals and signs",
    '13': "Hazardous parking",
    '14': "Lights",
    '15': "Brakes",
    '16': "Other equipment",
    '17': "Other hazardous violation",
    '18': "Other than driver (or pedestrian)",
    '19': "Unknown",
    '20': "Unknown",
    '21': "Unsafe starting or backing",
    '22': "Other improper driving",
    '23': 'Pedestrian or "other" under the influence of alcohol or drug',
    '24': "Fell asleep",
    '00': "Unknown",
    '- ' : "Unknown"
}

df['PRIMARY_COLLISION_FACTOR'] = df['PCF_VIOL_CATEGORY'].apply(lambda x: PCF_Decode[x])
df['PRIMARY_COLLISION_FACTOR'].head()

0          Pedestrian violation
1          Pedestrian violation
2       Pedestrian right of way
3       Pedestrian right of way
4    Unsafe starting or backing
Name: PRIMARY_COLLISION_FACTOR, dtype: object

## Hit and Run

In [13]:
Hit_and_Run_Decode = {
    'N' : "No",
    'F' : 'Hit and run (felony)',
    'M' : 'Hit and run (misdemeanor)'
}

df['HIT_AND_RUN'] = df['HIT_AND_RUN'].apply(lambda x: Hit_and_Run_Decode[x])
df['HIT_AND_RUN'].head()

0                      No
1                      No
2                      No
3    Hit and run (felony)
4                      No
Name: HIT_AND_RUN, dtype: object

## Pedestrian Action

In [14]:
Ped_Action_Decode = {
    'B' : 'Crossing, in crosswalk at intersection',
    'C' : 'Crossing, in crosswalk not at intersection',
    'D' : 'Crossing, not in crosswalk',
    'E' : 'In road, including shoulder',
    'F' : 'Not in road',
    'G' : 'Approaching or leaving school bus',
    '-' : 'Unknown'
}

df['PED_ACTION'] = df['PED_ACTION'].apply(lambda x: Ped_Action_Decode[x])
df['PED_ACTION'].head()

0                Crossing, not in crosswalk
1                Crossing, not in crosswalk
2    Crossing, in crosswalk at intersection
3    Crossing, in crosswalk at intersection
4                Crossing, not in crosswalk
Name: PED_ACTION, dtype: object

## Road surface

In [15]:
Road_Surface_Decode = {
    'A' : 'Dry',
    'B' : 'Wet',
    'C' : 'Snowy or Icy',
    'D' : 'Slippery (Muddy, Oily, etc.)',
    '-' : 'Unknown'
}

df['ROAD_SURFACE'] = df['ROAD_SURFACE'].apply(lambda x: Road_Surface_Decode[x])
df['ROAD_SURFACE'].head()

0    Dry
1    Wet
2    Wet
3    Dry
4    Dry
Name: ROAD_SURFACE, dtype: object

## Merge road condition

In [16]:
def merge_road_conditions(S):
    
    Road_Cond_Decode = {
        'A' : 'Holes or deep ruts',
        'B' : 'Loose material on roadway',
        'C' : 'Obstruction on roadway',
        'D' : 'Construction or repair zone',
        'E' : 'Reduced roadway width',
        'F' : 'Flooded',
        'G' : 'Other',
        'H' : 'No unusual condition',
        '-' : 'Unknown'
    }
    
    out = Road_Cond_Decode[S['ROAD_COND_1']]
    
    if S['ROAD_COND_2'] != '-':
        out = ', '.join([out, Road_Cond_Decode[S['ROAD_COND_2']]])
    
    return out

In [17]:
df['ROAD_COND'] = df[['ROAD_COND_1','ROAD_COND_2']].apply(merge_road_conditions,axis=1)
df['ROAD_COND'].head()

0    No unusual condition
1    No unusual condition
2    No unusual condition
3    No unusual condition
4    No unusual condition
Name: ROAD_COND, dtype: object

## Lighting

In [18]:
Lighting_Decode = {
    'A' : 'Daylight',
    'B' : 'Dusk or Dawn',
    'C' : 'Dark - Street lights',
    'D' : 'Dark - No street lights',
    'E' : 'Dark - Street lights not functioning',
    '-' : 'Unknown'
}

df['LIGHTING'] = df['LIGHTING'].apply(lambda x: Lighting_Decode[x])
df['LIGHTING'].head()

0                   Daylight
1       Dark - Street lights
2       Dark - Street lights
3    Dark - No street lights
4                   Daylight
Name: LIGHTING, dtype: object

## Alcohol Involved

In [30]:
df.replace(to_replace='Y',value=True,inplace=True)
df['ALCOHOL_INVOLVED'].fillna(value=False,inplace=True)
print df['ALCOHOL_INVOLVED'].value_counts()
df['ALCOHOL_INVOLVED'].iloc[10:15]

False    9340
True     1119
Name: ALCOHOL_INVOLVED, dtype: int64


10    False
11    False
12     True
13    False
14    False
Name: ALCOHOL_INVOLVED, dtype: bool

## Pedestrians Killed

In [20]:
df['COUNT_PED_KILLED'] = df['COUNT_PED_KILLED'].apply(int)
df['COUNT_PED_KILLED'].value_counts()

0    10228
1      231
Name: COUNT_PED_KILLED, dtype: int64

## Keep columns

In [21]:
keepcols = [
    'ACCIDENT_YEAR',
    'COLLISION_DATE',
    'COLLISION_TIME',
    'DAY_OF_WEEK',
    'location',
    'WEATHER',
    'PRIMARY_COLLISION_FACTOR',
    'HIT_AND_RUN',
    'PED_ACTION',
    'ROAD_SURFACE',
    'ROAD_COND',
    'LIGHTING',
    'ALCOHOL_INVOLVED',
    'COUNT_PED_KILLED',
    'COUNT_PED_INJURED',
    'y',
    'x'
    ]

df = df[keepcols]

In [22]:
# Rename columns
df.rename(columns={
        'ACCIDENT_YEAR': 'year', 
        'COLLISION_DATE': 'date',
        'COLLISION_TIME' : 'time',
        'DAY_OF_WEEK' : 'dayofweek'
    },inplace=True)
pass

## Check output

In [23]:
df.head()

,year,date,time,dayofweek,location,WEATHER,PRIMARY_COLLISION_FACTOR,HIT_AND_RUN,PED_ACTION,ROAD_SURFACE,ROAD_COND,LIGHTING,ALCOHOL_INVOLVED,COUNT_PED_KILLED,COUNT_PED_INJURED,y,x
0,2003,2003-01-13,17:00:00,Monday,"JACKSON ST and GRANT AV, 77.0 ft east",Clear,Pedestrian violation,No,"Crossing, not in crosswalk",Dry,No unusual condition,Daylight,False,0,1,37.796035,-122.406684
1,2003,2003-01-09,20:30:00,Thursday,"TARAVAL ST and 19TH AV, 64.0 ft west",Raining,Pedestrian violation,No,"Crossing, not in crosswalk",Wet,No unusual condition,Dark - Street lights,False,0,1,37.743050,-122.475712
2,2003,2003-01-10,06:40:00,Friday,"N 7TH ST and MARKET ST, 20.0 ft north",Cloudy,Pedestrian right of way,No,"Crossing, in crosswalk at intersection",Wet,No unusual condition,Dark - Street lights,False,0,1,37.780466,-122.412557
3,2003,2003-01-01,18:30:00,Wednesday,"FRANKLIN ST and POST ST, at intersection",Clear,Pedestrian right of way,Hit and run (felony),"Crossing, in crosswalk at intersection",Dry,No unusual condition,Dark - No street lights,False,0,1,37.786432,-122.423233
4,2003,2003-01-10,10:05:00,Friday,"CLEMENTINA ST and 4TH ST, 90.0 ft west",Cloudy,Unsafe starting or backing,No,"Crossing, not in crosswalk",Dry,No unusual condition,Daylight,False,0,1,37.782482,-122.401713
